# Example: scenarios for TAMU grid


In [67]:
import pandas as pd
import numpy as np

from powerscenarios.parser import Parser
from powerscenarios.grid import Grid


# from numpy import linalg
# import matplotlib.pyplot as plt

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)


import time

In [2]:
# plotting (optional)
import cufflinks as cl

cl.go_offline()

# parse TAMU grid .aux files

In [3]:
# choose TAMU grid
# grid files can be downloaded from:
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/

grid_name = "ACTIVSg200"  # TAMU 200 bus case
# grid_name = "ACTIVSg2000"  # TAMU 2000 bus case
# grid_name = 'ACTIVSg10k' # TAMU 10000 bus case

# path to .aux file (TAMU grids) obtained from e.g.
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/activsg200/
data_dir = "../data/grid-data/"
aux_file_name = data_dir + grid_name + "/" + grid_name + ".aux"

# parse original .aux file and return dataframes for buses, generators, and wind generators
# here, we need .aux files because those are the only ones with Latitute/Longitude information
parser = Parser()
bus_df, gen_df, wind_gen_df = parser.parse_tamu_aux(aux_file_name)

# see what you got
print("bus_df:")
bus_df.head()
print("gen_df:")
gen_df.head()
print("wind_gen_df:")
wind_gen_df.head()

bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6


gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.53,1.36,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.53,1.36,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.53,1.36,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.53,1.36,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
4,53,1,9.07,2.72,1.0,Coal,53_Coal_1,RANTOUL25,40.312222,-88.159444,6


wind_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7


## instantiate Grid class

In [4]:
# to instantiate a grid we need: name, bus, generator, and wind generator dataframes from Parser
# really, we only wind generator, might change in the future
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=0)


ACTIVSg200 grid info: 

 number of buses: 200
 number of generators: 49
 number of wind generators: 6
 number of solar generators: 0
 total generator capacity: 3602.84 MW
 wind capacity/penetration: 699.60 MW / 19.42%
 solar capacity/penetration: 0.00 MW / 0.00%


## (optional) change_wind_penetration method

In [5]:
# ?grid.change_wind_penetration

## retrieve_wind_sites method

In [6]:
# ?grid.retrieve_wind_sites

In [7]:
# retrieve wind sites matching current wind penetration
# currently, uses AWS (later will implement "source" option to choose from AWS or local data )
# uses pywtk_api

# retrieve wind sites (wind_sites are initially set to empty df )
grid
grid.retrieve_wind_sites()
grid
grid.wind_sites.head()

Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=0)

Retrieving wind sites ...
Done


Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=50)

,SiteID,Capacity,Point,Latitude,Longitude,BusNum,GenUID
0,54007,16.0,POINT (-88.02514600000001 40.458515),40.458515,-88.025146,65,65_Wind_1
1,54196,16.0,POINT (-88.02273599999999 40.476978),40.476978,-88.022736,65,65_Wind_1
2,54008,16.0,POINT (-88.000885 40.456665),40.456665,-88.000885,65,65_Wind_1
3,53833,16.0,POINT (-88.02758799999999 40.440052),40.440052,-88.027588,65,65_Wind_1
4,54197,16.0,POINT (-87.998474 40.475128),40.475128,-87.998474,65,65_Wind_1


## make_tables method (uses internal retrieve_wtk_data method)

In [8]:
grid.make_tables(
    actuals_start=pd.Timestamp("2007-01-01 00:00:00", tz="utc"),
    actuals_end=pd.Timestamp("2007-12-31 23:55:00", tz="utc"),
    scenarios_start=pd.Timestamp("2008-01-01 00:00:00", tz="utc"),
    scenarios_end=pd.Timestamp("2013-12-31 23:55:00", tz="utc"),
    source="AWS",
)

Retrieving WTK data ...
Done
Retrieving WTK data ...
Done


## change actuals year as needed

In [9]:
# for actuals, make year you want
grid.actuals.index = grid.actuals.index.map(lambda t: t.replace(year=2020))
# see what you got
print("\nactuals_df:")
grid.actuals.head()
print("\nscenarios_df:")
grid.scenarios.head()


actuals_df:


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower
IssueTime,,,,,,,
2020-01-01 00:00:00+00:00,142.094736,99.000001,198.000002,1.7,150.0,100.5,691.294739
2020-01-01 00:05:00+00:00,140.804726,99.000001,198.000002,1.7,150.0,100.5,690.004728
2020-01-01 00:10:00+00:00,142.361249,99.000001,198.000002,1.7,150.0,100.5,691.561251
2020-01-01 00:15:00+00:00,145.972502,99.000001,198.000002,1.7,150.0,100.5,695.172504
2020-01-01 00:20:00+00:00,150.399995,99.000001,198.000002,1.7,150.0,100.5,699.599997



scenarios_df:


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower,Deviation
IssueTime,,,,,,,,
2008-01-01 00:00:00+00:00,1.459344,0.668133,0.583035,-0.024500,0.877288,1.845110,96.746560,5.408411
2008-01-01 00:05:00+00:00,1.601367,-0.445220,1.269019,0.091567,1.525665,3.528319,102.154971,7.570718
2008-01-01 00:10:00+00:00,1.097983,-0.016742,1.678258,0.294259,3.257260,5.579959,109.725689,11.890977
2008-01-01 00:15:00+00:00,1.172363,2.034480,3.025435,0.001593,4.814818,3.355411,121.616666,14.404100
2008-01-01 00:20:00+00:00,1.237113,2.234493,2.933309,0.000000,5.278723,6.783124,136.020766,18.466761


## generate_wind_scenarios

In [73]:
# time period for which to generate scenarios

# a few timestamps timestamp
sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),] 
#sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),pd.Timestamp("2020-07-01 00:20:00+0000", tz="UTC")] 

# #range
# sim_timestamps = pd.date_range(
#     start=pd.Timestamp("2020-07-01 00:00:00+0000", tz="UTC"), end=pd.Timestamp("2020-07-01 00:10:00+0000", tz="UTC"), freq="5min"
# )


# other parameters
#sampling_method="monte carlo"
sampling_method="importance"
n_scenarios = 10
n_periods = 6



all_scenarios_df = pd.DataFrame()
all_weights_df = pd.DataFrame()

for sim_timestamp in sim_timestamps:
    print("sim_timestamp = {}".format(sim_timestamp))
    random_seed = np.random.randint(2 ** 31 - 1)
    #random_seed = 594081473
    print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.1, 0.9, 1.0],
        sampling_method=sampling_method,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        # random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_weights_df=pd.concat([all_weights_df,weights_df])
        
all_scenarios_df
all_weights_df



sim_timestamp = 2020-07-01 00:15:00+00:00
random_seed = 320297024


65_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                      
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00   33.1197   
                                      2020-07-01 00:20:00+00:00   35.4053   
                                      2020-07-01 00:25:00+00:00   37.2813   
                                      2020-07-01 00:30:00+00:00   38.7384   
                                      2020-07-01 00:35:00+00:00   39.6847   
...                                                                   ...   
                          10          2020-07-01 00:20:00+00:00   24.3359   
                                      2020-07-01 00:25:00+00:00    24.131   
                                      2020-07-01 00:30:00+00:00   24.0396   
                                      2020-07-01 00:35:00+00:00   24.6736   
                                      2020-07-01 00:40:00+00:00   24.6197   

                                                                104_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    12.4853   
                                      2020-07-01 00:20:00+00:00    12.4538   
                                      2020-07-01 00:25:00+00:00    11.3955   
                                      2020-07-01 00:30:00+00:00     11.155   
                                      2020-07-01 00:35:00+00:00    10.1868   
...                                                                    ...   
                          10          2020-07-01 00:20:00+00:00    2.93223   
                                      2020-07-01 00:25:00+00:00          0   
                                      2020-07-01 00:30:00+00:00          0   
                                      2020-07-01 00:35:00+00:00          0   
                                      2020-07-01 00:40:00+00:00          0   

                                                                105_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    30.3497   
                                      2020-07-01 00:20:00+00:00    30.2755   
                                      2020-07-01 00:25:00+00:00    28.2976   
                                      2020-07-01 00:30:00+00:00    28.0837   
                                      2020-07-01 00:35:00+00:00    26.8224   
...                                                                    ...   
                          10          2020-07-01 00:20:00+00:00    11.4497   
                                      2020-07-01 00:25:00+00:00          0   
                                      2020-07-01 00:30:00+00:00          0   
                                      2020-07-01 00:35:00+00:00          0   
                                      2020-07-01 00:40:00+00:00          0   

                                                                114_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-07-01 00:15:00+00:00 1           2020-07-01 00:15:00+00:00    1.24115   
                                      2020-07-01 00:20:00+00:00    1.33188   
                                      2020-07-01 00:25:00+00:00    1.49707   
                                      2020-07-01 00:30:00+00:00        1.7   
                                      2020-07-01 00:35:00+00:00    1.59659   
...                                                                    ...   
                          10          2020-07-01 00:20:00+00:00    1.18193   
                                      2020-07-01 00:25:00+00:00   0.904164   
                                      2020-07-01 00:30:00+00:00   0.681776   
                                      2020-07-01 00:35:00+00:00   0.529122   
                                      2020-07-01 00:40:00+00:00   0.442893   

                 

,1,2,3,4,5,6,7,8,9,10
2020-07-01 00:15:00+00:00,2.12927,0.337567,0.255777,0.170555,0.348728,0.401349,0.160297,0.768855,3.28665,0.237531


# plot 

In [74]:
# choose sim_timestamp for which to plot all scenarios
sim_timestamp = sim_timestamps[0]

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)
plot_df = pd.DataFrame(index=timestamps, columns=range(1, n_scenarios + 1),)
for scenario_nr in range(1, n_scenarios + 1):
    s = generated_scenarios_df.loc[(sim_timestamp, scenario_nr,)].sum(axis=1)
    s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

    plot_df[scenario_nr] = s
plot_df.iplot()

# save (for single period, can drop period_timestamp)

In [40]:
df = generated_scenarios_df.copy()
# if scenarios are single period, we can drop period_timestamp index level
if n_periods == 1:
    df.index=df.index.droplevel("period_timestamp")


filename = './scenarios.csv'

print("\nsaving generated_scenarios_df to {}".format(filename))
df
df.to_csv(filename)



# take actuals corresponding to scenarios
df=grid.actuals.loc[sim_timestamps].copy()
df.index=df.index.rename("sim_timestamp")

filename = './actuals.csv'

print("\nsaving actuals to {}".format(filename))
df
df.to_csv(filename)



saving generated_scenarios_df to ./scenarios.csv


65_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                      
2020-01-01 00:15:00+00:00 1           2020-01-01 00:15:00+00:00   142.361   
                                      2020-01-01 00:20:00+00:00   142.361   
                                      2020-01-01 00:25:00+00:00   142.361   
                                      2020-01-01 00:30:00+00:00   142.361   
                                      2020-01-01 00:35:00+00:00   142.361   
...                                                                   ...   
                          10          2020-01-01 00:20:00+00:00   142.361   
                                      2020-01-01 00:25:00+00:00   142.361   
                                      2020-01-01 00:30:00+00:00   142.361   
                                      2020-01-01 00:35:00+00:00   142.361   
                                      2020-01-01 00:40:00+00:00   142.361   

                                                                104_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-01-01 00:15:00+00:00 1           2020-01-01 00:15:00+00:00         99   
                                      2020-01-01 00:20:00+00:00         99   
                                      2020-01-01 00:25:00+00:00         99   
                                      2020-01-01 00:30:00+00:00         99   
                                      2020-01-01 00:35:00+00:00         99   
...                                                                    ...   
                          10          2020-01-01 00:20:00+00:00         99   
                                      2020-01-01 00:25:00+00:00         99   
                                      2020-01-01 00:30:00+00:00         99   
                                      2020-01-01 00:35:00+00:00         99   
                                      2020-01-01 00:40:00+00:00         99   

                                                                105_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-01-01 00:15:00+00:00 1           2020-01-01 00:15:00+00:00        198   
                                      2020-01-01 00:20:00+00:00        198   
                                      2020-01-01 00:25:00+00:00        198   
                                      2020-01-01 00:30:00+00:00        198   
                                      2020-01-01 00:35:00+00:00        198   
...                                                                    ...   
                          10          2020-01-01 00:20:00+00:00        198   
                                      2020-01-01 00:25:00+00:00        198   
                                      2020-01-01 00:30:00+00:00        198   
                                      2020-01-01 00:35:00+00:00        198   
                                      2020-01-01 00:40:00+00:00        198   

                                                                114_Wind_1  \
sim_timestamp             scenario_nr period_timestamp                       
2020-01-01 00:15:00+00:00 1           2020-01-01 00:15:00+00:00        1.7   
                                      2020-01-01 00:20:00+00:00        1.7   
                                      2020-01-01 00:25:00+00:00        1.7   
                                      2020-01-01 00:30:00+00:00        1.7   
                                      2020-01-01 00:35:00+00:00        1.7   
...                                                                    ...   
                          10          2020-01-01 00:20:00+00:00        1.7   
                                      2020-01-01 00:25:00+00:00        1.7   
                                      2020-01-01 00:30:00+00:00        1.7   
                                      2020-01-01 00:35:00+00:00        1.7   
                                      2020-01-01 00:40:00+00:00        1.7   

                 


saving actuals to ./actuals.csv


,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower
sim_timestamp,,,,,,,
2020-01-01 00:15:00+00:00,145.972502,99.000001,198.000002,1.7,150.0,100.5,695.172504


<IPython.core.display.Javascript object>

# Scratch

In [21]:
grid.scenarios

,65_Wind_1,104_Wind_1,105_Wind_1,114_Wind_1,115_Wind_1,147_Wind_1,TotalPower,Deviation
IssueTime,,,,,,,,
2008-01-01 00:00:00+00:00,1.459344,0.668133,0.583035,-0.024500,0.877288,1.845110,96.746560,5.408411
2008-01-01 00:05:00+00:00,1.601367,-0.445220,1.269019,0.091567,1.525665,3.528319,102.154971,7.570718
2008-01-01 00:10:00+00:00,1.097983,-0.016742,1.678258,0.294259,3.257260,5.579959,109.725689,11.890977
2008-01-01 00:15:00+00:00,1.172363,2.034480,3.025435,0.001593,4.814818,3.355411,121.616666,14.404100
2008-01-01 00:20:00+00:00,1.237113,2.234493,2.933309,0.000000,5.278723,6.783124,136.020766,18.466761
...,...,...,...,...,...,...,...,...
2013-12-31 23:30:00+00:00,0.000000,0.000432,0.005397,0.002113,0.084650,0.592468,3.752755,0.685059
2013-12-31 23:35:00+00:00,0.000000,0.000429,0.007042,0.001457,0.078459,0.659893,4.437814,0.747280
2013-12-31 23:40:00+00:00,0.000000,0.003806,0.018556,0.002599,0.072297,0.657600,5.185094,0.754858


In [25]:
power_quantiles=[0.0, 0.1, 0.9, 1.0]

power_bins = pd.qcut(
    grid.scenarios["TotalPower"].iloc[:-n_periods], q=power_quantiles,
)

power_bins

IssueTime
2008-01-01 00:00:00+00:00    (11.458, 689.156]
2008-01-01 00:05:00+00:00    (11.458, 689.156]
2008-01-01 00:10:00+00:00    (11.458, 689.156]
2008-01-01 00:15:00+00:00    (11.458, 689.156]
2008-01-01 00:20:00+00:00    (11.458, 689.156]
                                   ...        
2013-12-31 23:00:00+00:00     (-0.001, 11.458]
2013-12-31 23:05:00+00:00     (-0.001, 11.458]
2013-12-31 23:10:00+00:00     (-0.001, 11.458]
2013-12-31 23:15:00+00:00     (-0.001, 11.458]
2013-12-31 23:20:00+00:00     (-0.001, 11.458]
Name: TotalPower, Length: 631289, dtype: category
Categories (3, interval[float64]): [(-0.001, 11.458] < (11.458, 689.156] < (689.156, 699.6]]